In [1]:
from omegaconf import OmegaConf
import argparse
import yaml

parser = argparse.ArgumentParser(description="hello")
parser.add_argument('--local_rank', default=-1, type=int, help='node rank for distributed training')
parser.add_argument('--cfg_path', default=None, type=str, help='node rank for distributed training')
#args = parser.parse_args()

args = parser.parse_args(['--local_rank', '0', '--cfg_path', 'cfgs/finetune/finetune_fengwu.yaml'])

with open(args.cfg_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Convert to omegaconf.DictConfig
config = OmegaConf.create(yaml_data)
config_deepspeed = OmegaConf.create({"deepspeed":vars(args)})
config = OmegaConf.merge(config, config_deepspeed)
cfg = config


In [2]:
import numpy as np
from dataset.dataset_fengwu import Replay_buffer_fengwu
import os

base_path = cfg.data.npy_name
print(base_path)
print(cfg.data.shape)
data_ori = np.memmap(base_path, dtype = 'float32',mode = 'c', shape = tuple(cfg.data.shape) , order = 'C')



buffer_file = os.path.join(cfg.data.buffer_folder,cfg.logger.name+'.npy')
print(buffer_file)
print(cfg.data.train_range)

Replay_Buffer = Replay_buffer_fengwu(
        data=data_ori, train_range=cfg.data.train_range, buffer_file= buffer_file,
        buffer_size=cfg.data.buffer_size, shape = cfg.data.shape,
    )
dataset = Replay_Buffer.build_dataset(size=6400)



/dev/shm/store/original_dataset/dataset.npy
[58436, 68, 32, 64]
/dev/shm/store/buffer/swin-multi-finetune.npy
[0, 56000]
20000 20000


In [3]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [4]:
s = next(iter(dataloader))

In [5]:
s[1].shape

torch.Size([20, 1, 68, 32, 64])

In [6]:
import importlib
model_path = cfg.model.model_path
model_type = cfg.model.model_type
model_module = importlib.import_module(model_path)
model_class = getattr(model_module, model_type)
model = model_class(**cfg.model.param)

/home/gaoziyi/miniconda3/envs/train-weather/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
